# Model Comparison

In diesem Notebook wird die Performance der drei Modelle miteinander verglichen, um anschliessend das beste Modelle für die Anwendung auszuwählen.

## 1. Modelle und Daten laden

Zuerst werden die trainierten Modelle und der bereinigte Datensatz geladen.

In [22]:
import pandas as pd
from joblib import load
from data_loader import load_data
from sklearn.metrics import mean_squared_error

# Modelle laden
rf = load("../Models/random_forest_model.joblib")
gd = load("../Models/gradient_boosting_model.joblib")
lr = load("../Models/linear_regression_model.joblib")

# Lade den bereinigten Datensatz
df = pd.read_csv("../data/life_expectancy_cleaned.csv")


# Daten laden
X_train, X_test, y_train, y_test = load_data()


## 2. Root Mean Squared Error (RMSE) und R^2
Um eine Model Comparison durchzuführen, schauen wir uns zwei Hauptmetriken an: Root Mean Squared Error (RMSE) und R^2.

- RMSE: Diese Metrik gibt uns den durchschnittlichen Fehler, den unser Modell macht. Eine niedrigere RMSE bedeutet, dass unser Modell im Durchschnitt weniger Fehler macht, was gut ist.
- R^2: Diese Metrik gibt uns eine Vorstellung davon, wie viel der Variation in den Daten durch unser Modell erklärt wird. Ein höherer R^2-Wert (näher an 1) bedeutet, dass unser Modell einen größeren Anteil der Variation erklärt, was ebenfalls gut ist.

Ein idealer Algorithmus hätte also eine niedrige RMSE und einen hohen R^2.

In [26]:
# Gradient Boosting Metriken berechnen
gd_preds = gd.predict(X_test)
gd_rmse = mean_squared_error(y_test, gd_preds, squared=False)
gd_r2 = gd.score(X_test, y_test)

# Lineare Regression Metriken berechnen
lr_preds = lr.predict(X_test)
lr_rmse = mean_squared_error(y_test, lr_preds, squared=False)
lr_r2 = lr.score(X_test, y_test)

# Random Forest Metriken berechnen
rf_preds = rf.predict(X_test)
rf_rmse = mean_squared_error(y_test, rf_preds, squared=False)
rf_r2 = rf.score(X_test, y_test)

# Zunächst erstellen wir eine leere DataFrame, um die Metriken zu speichern
model_comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Linear Regression', 'Gradient Boosting'],
    'RMSE': [rf_rmse, lr_rmse, gd_rmse],
    'R-Squared': [rf_r2, lr_r2, gd_r2]
})

# Sortieren nach RMSE
model_comparison.sort_values('RMSE', ascending=True, inplace=True)

print(model_comparison)


               Model          RMSE     R-Squared
0      Random Forest  3.094892e-02  9.692944e-01
2  Gradient Boosting  4.002273e-02  9.486501e-01
1  Linear Regression  6.711573e+10 -1.444026e+23


## 3. Cross Validation and Mean Absoulte Error

Um eine noch umfassendere Bewertung der Modelle zu ermöglichen, können wir zusätzliche Metriken und Techniken einsetzen. Eine davon ist die Kreuzvalidierung (Cross-Validation), bei der das Datenset in k Gruppen oder "Folds" aufgeteilt wird. Das Modell wird dann k Mal trainiert und getestet, wobei in jeder Runde eine andere Gruppe als Testset und die restlichen Gruppen als Trainingssatz verwendet werden. Diese Technik gibt uns eine robustere Einschätzung der Modellleistung, da sie das Modell auf verschiedenen Teilen des Datensatzes testet.

Außerdem können wir den durchschnittlichen absoluten Fehler (Mean Absolute Error - MAE) betrachten, der den durchschnittlichen absoluten Unterschied zwischen den tatsächlichen und den vorhergesagten Werten misst.

In [29]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error

# Kreuzvalidierung und MAE für Random Forest
rf_cv_scores = cross_val_score(rf, X_test, y_test, cv=5)
rf_mae = mean_absolute_error(y_test, rf_preds)

# Kreuzvalidierung und MAE für Linear Regression
lr_cv_scores = cross_val_score(lr, X_test, y_test, cv=5)
lr_mae = mean_absolute_error(y_test, lr_preds)

# Kreuzvalidierung und MAE für Gradient Boosting
gb_cv_scores = cross_val_score(gd, X_test, y_test, cv=5)
gb_mae = mean_absolute_error(y_test, gd_preds)

# Füge die neuen Metriken zur Vergleichstabelle hinzu
model_comparison['Cross Val Score (mean)'] = [rf_cv_scores.mean(), lr_cv_scores.mean(), gb_cv_scores.mean()]
model_comparison['MAE'] = [rf_mae, lr_mae, gb_mae]

print(model_comparison)


/Users/dinopelesevic/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dinopelesevic/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dinopelesevic/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_validation.py:686: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/Users/dinopelesevic/opt/anaconda3/lib/python3.9/site-packages/sklearn

               Model          RMSE     R-Squared  Cross Val Score (mean)  \
0      Random Forest  3.094892e-02  9.692944e-01            9.216048e-01   
2  Gradient Boosting  4.002273e-02  9.486501e-01           -4.496573e+24   
1  Linear Regression  6.711573e+10 -1.444026e+23            9.164013e-01   

            MAE  
0  1.975505e-02  
2  2.767806e+09  
1  2.950472e-02  


## 4. Mean Squared Logarithmic Error, Median Absolute Error & Explained Variance Score
Als nächstes verwenden wir noch einige weitere Metriken, um die Modelle miteinander zu vergleichen:

- Mean Squared Logarithmic Error (MSLE): Diese Metrik ist besonders nützlich, wenn exponentielle Wachstumsprozesse vorhergesagt werden sollen. Sie ist weniger empfindlich gegenüber großen Fehlern und legt mehr Gewicht auf kleine Fehler.
- Median Absolute Error: Diese Metrik ähnelt dem Mean Absolute Error, aber anstatt den Durchschnitt der absoluten Fehler zu berechnen, nimmt sie den Median. Das macht sie robuster gegenüber Ausreißern.
- Explained Variance Score: Diese Metrik misst den Anteil der Varianz, der durch das Modell erklärt wird. Eine Punktzahl von 1.0 zeigt an, dass das Modell perfekt die Zielvariable vorhersagt.

In [31]:
import numpy as np
from sklearn.metrics import mean_squared_log_error, median_absolute_error, explained_variance_score

# Funktion, um die Wurzel des MSLE zu berechnen
def rmsle(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# RMSLE, Median Absolute Error und Explained Variance Score für Random Forest
rf_rmsle = rmsle(y_test, rf_preds)
rf_medae = median_absolute_error(y_test, rf_preds)
rf_evs = explained_variance_score(y_test, rf_preds)

# RMSLE, Median Absolute Error und Explained Variance Score für Linear Regression
lr_rmsle = rmsle(y_test, lr_preds)
lr_medae = median_absolute_error(y_test, lr_preds)
lr_evs = explained_variance_score(y_test, lr_preds)

# RMSLE, Median Absolute Error und Explained Variance Score für Gradient Boosting
gb_rmsle = rmsle(y_test, gd_preds)
gb_medae = median_absolute_error(y_test, gd_preds)
gb_evs = explained_variance_score(y_test, gd_preds)

# Füge die neuen Metriken zur Vergleichstabelle hinzu
model_comparison['RMSLE'] = [rf_rmsle, lr_rmsle, gb_rmsle]
model_comparison['Median AE'] = [rf_medae, lr_medae, gb_medae]
model_comparison['Explained Variance Score'] = [rf_evs, lr_evs, gb_evs]

print(model_comparison)


               Model          RMSE     R-Squared  Cross Val Score (mean)  \
0      Random Forest  3.094892e-02  9.692944e-01            9.216048e-01   
2  Gradient Boosting  4.002273e-02  9.486501e-01           -4.496573e+24   
1  Linear Regression  6.711573e+10 -1.444026e+23            9.164013e-01   

            MAE     RMSLE  Median AE  Explained Variance Score  
0  1.975505e-02  0.019816   0.011812              9.694658e-01  
2  2.767806e+09  1.139756   0.014524             -1.441570e+23  
1  2.950472e-02  0.025537   0.022536              9.486780e-01  


## 5. Mean Absolute Percentage Error & R^2 adjusted
Zuletzt wollen wir noch den Mean Absolute Percentage Error und den R^2 adjusted berechnen. 

- Mean Absolute Percentage Error (MAPE): Diese Metrik gibt den durchschnittlichen prozentualen Fehler in Bezug auf die tatsächlichen Werte an. Sie ist besonders nützlich, wenn du die Fehler relativ zu den tatsächlichen Werten betrachten möchtest.
- R^2 adjusted: Diese Metrik passt den R^2-Wert an die Anzahl der Prädiktoren in dem Modell an. Dies ist besonders nützlich, wenn du Modelle mit unterschiedlicher Anzahl von Prädiktoren vergleichst.

Davor muss aber noch die Länge der Testdaten und Vorhersagen gleich sein:

In [35]:
#Länge der Testdaten und der Vorhersagen vergleichen
print(y_test.shape)
print(rf_preds.shape)

# y_test in ein Array umwandeln
y_test = y_test.squeeze()

# Länge der Testdaten und der Vorhersagen vergleichen
print(y_test.shape)


(588,)
(588,)
(588,)


Anschliessend können Mean Absolute Percentage Error und R^2 adjusted berechnet werden:

In [37]:
# Funktion zur Berechnung des MAPE
def mape(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

# Funktion zur Berechnung des R^2 adjusted
def r2_adjusted(r2, n, p):
    return 1 - (1 - r2) * (n - 1) / (n - p - 1)

# MAPE und R^2 adjusted für Random Forest
rf_mape = mape(y_test, rf_preds)
rf_r2_adj = r2_adjusted(rf_r2, len(y_test), X_test.shape[1])

# MAPE und R^2 adjusted für Linear Regression
lr_mape = mape(y_test, lr_preds)
lr_r2_adj = r2_adjusted(lr_r2, len(y_test), X_test.shape[1])

# MAPE und R^2 adjusted für Gradient Boosting
gb_mape = mape(y_test, gd_preds)
gb_r2_adj = r2_adjusted(gd_r2, len(y_test), X_test.shape[1])

# Füge die neuen Metriken zur Vergleichstabelle hinzu
model_comparison['MAPE'] = [rf_mape, lr_mape, gb_mape]
model_comparison['R-Squared Adjusted'] = [rf_r2_adj, lr_r2_adj, gb_r2_adj]

print(model_comparison)


               Model          RMSE     R-Squared  Cross Val Score (mean)  \
0      Random Forest  3.094892e-02  9.692944e-01            9.216048e-01   
2  Gradient Boosting  4.002273e-02  9.486501e-01           -4.496573e+24   
1  Linear Regression  6.711573e+10 -1.444026e+23            9.164013e-01   

            MAE     RMSLE  Median AE  Explained Variance Score          MAPE  \
0  1.975505e-02  0.019816   0.011812              9.694658e-01  3.820506e+00   
2  2.767806e+09  1.139756   0.014524             -1.441570e+23  4.430180e+11   
1  2.950472e-02  0.025537   0.022536              9.486780e-01  5.583414e+00   

   R-Squared Adjusted  
0        9.519355e-01  
2       -2.260382e+23  
1        9.196202e-01  


## 6. Modellwahl und Fazit

In diesem Kapitel wollen wir die Resultat der Model Comparison analysieren. Beginnen wir dafür mit den Resultaten des Random Forest Modells:

### 6.1 Random Forest: 

- **RMSE**:                      3.094892e-02. Das bedeutet, dass das Modell im Durchschnitt eine Abweichung von etwa 3.1% vom tatsächlichen Wert hat. Dies ist ein ziemlich guter Wert und deutet auf eine hohe Genauigkeit des Modells hin.

- **R-Squared**:                 9.692944e-01 (96.93%). Dies bedeutet, dass 96.93% der Variationen in den Daten durch das Modell erklärt werden können. Dies ist ein ausgezeichneter Wert.

- **Cross Val Score (mean)**:    9.216048e-01 (92.16%). Dies deutet darauf hin, dass das Modell konstant hohe Genauigkeiten über verschiedene Aufteilungen der Trainingsdaten liefert.

- **MAE**:                       1.975505e-02 (1.98%). Dies deutet darauf hin, dass das Modell im Durchschnitt etwa 1.98% vom tatsächlichen Wert entfernt liegt. Dies ist ebenfalls ein sehr guter Wert.

- **RMSLE**:                     0.019816. Dies ist der Logarithmische Mittlere Quadratische Fehler. Je näher dieser Wert an 0 ist, desto besser das Modell.

- **Median AE**:                 0.011812. Dies ist der mediane absolute Fehler. Ein niedrigerer Wert ist besser.

- **Explained Variance Score**:  9.694658e-01 (96.95%). Dies ist ein weiteres Maß dafür, wie gut das Modell die Variationen in den Daten erklärt.

- **MAPE**                       3.820506e+00 (3.82%). Dies ist der mittlere absolute prozentuale Fehler. Ein niedrigerer Wert ist besser.

- **R-Squared Adjusted**:        9.519355e-01 (95.19%). Dies ist ein bereinigtes R-Squared, das die Anzahl der Prädiktoren im Modell berücksichtigt.

Wir sehen also, dass die Werte des Random Forest durchaus sinnvoll sind und das Modell gut performen sollte. Der Wert des R^2 ist zwar ein ausgezeichneter Wert, allerdings sind solche Werte immer mit Vorsicht zu geniessen. Ein hoher R^2-Wert kann auch ein Anzeichen für Überanpassung sein, insbesondere wenn das Modell eine große Anzahl von Features hat. In solchen Fällen könnte das Modell die Trainingsdaten "zu gut" lernen und möglicherweise schlecht auf neue, bisher ungesehene Daten reagieren.



### 6.2 Gradient Boosting

Der Gradient Boosting hat eine negative Cross-Validation-Score und eine sehr große MAE, RMSLE und MAPE. Dies könnte auf ein Overfitting des Modells hinweisen, oder dass das Modell Schwierigkeiten hat, die Komplexität der Daten anzupassen.



### 6.3 Linear Regression:

Die lineare Regression hat ein negatives R-Squared und einen sehr hohen RMSE. Dies könnte darauf hindeuten, dass das Modell Schwierigkeiten hat, die Daten anzupassen, oder dass es starke Nichtlinearitäten in den Daten gibt, die das Modell nicht berücksichtigt.

### Fazit

Auf Grundlage dieser Ergebnisse scheint das **Random Forest-Modell** die beste Leistung zu haben. Es hat die höchsten Werte für R-Squared und Cross Validation Score und die niedrigsten Fehlerwerte. Es scheint auch, dass es am besten in der Lage ist, die Variationen in den Daten zu erklären.

